In [ ]:


import random
class SudokuPuzzle:
    def __init__(self, filepath):
        self.grid = self.load_puzzle(filepath)
        self.domains = [[None for _ in range(9)] for _ in range(9)]
    
    """Load a Sudoku puzzle from a file and replace '?' with 0."""
    def load_puzzle(self, filepath):
        with open(filepath, 'r', encoding='utf-8-sig') as file:
            lines = [line.strip().replace('?', '0') for line in file]
        grid = []
        for line in lines:
            row = [int(num) for num in line.split(',')]
            grid.append(row)
        return grid

    """Display the Sudoku grid"""
    def display(self):
        for row in self.grid:
            print(" ".join(str(num) if num != 0 else '?' for num in row))

    """Output the Sudoku grid to a file, replacing 0 with '?'"""
    def output_to_file(self, filepath):
        with open(filepath, 'w') as file:
            for row in self.grid:
                line = ",".join(str(num) if num != 0 else '?' for num in row)
                file.write(line + "\n")

    """Check if all rows are valid (no duplicates except 0)"""
    def is_valid_rows(self):
        for row in self.grid:
            nums = [num for num in row if num != 0]
            if len(nums) != len(set(nums)):
                return False
        return True
    
    """Check if all columns are valid (no duplicates except 0)"""
    def is_valid_columns(self):
        for col in range(9):
            nums = [self.grid[row][col] for row in range(9) if self.grid[row][col] != 0]
            if len(nums) != len(set(nums)):
                return False
        return True
    
    """Check if all 3x3 blocks are valid (no duplicates except 0)"""
    def is_valid_blocks(self):
        for block_start_row in [0, 3, 6]:
            for block_start_col in [0, 3, 6]:
                if not self._is_valid_blocks_checker(block_start_row, block_start_col):
                    return False
        return True
    
    """Check if the entire Sudoku grid is valid"""
    def is_valid(self):
        return self.is_valid_rows() and self.is_valid_columns() and self.is_valid_blocks()
    
    """Helper function to check a single 3x3 block"""
    def _is_valid_blocks_checker(self, block_start_row, block_start_col):
        nums = []
        for i in range(3):
            for j in range(3):
                num = self.grid[block_start_row + i][block_start_col + j]
                if num != 0:
                    nums.append(num)
        return len(nums) == len(set(nums))
    
    """Set a digit in the grid at a specific row and column"""
    def set_digit(self, row, col, digit):
        self.grid[row][col] = digit

    """Set Domains for a puzzle"""
    def set_domains(self):
        row_domains = [self.get_set_from_row(row) for row in range(9)]
        column_domains = [self.get_set_from_column(col) for col in range(9)]
        block_domains = []
        for row in [0,3,6]:
            domain = []
            for col in [0,3,6]:
                domain.append(self.get_set_from_block(row, col))
            block_domains.append(domain)
        for row in range(9):
            for col in range(9):
                if self.grid[row][col] != 0:
                    continue
                not_domain = row_domains[row].union(column_domains[col]).union(block_domains[row//3][col//3])
                dom = set(range(1,10)).difference(not_domain)
                self.domains[row][col] = list(dom)

    """Display the Sudoku domain"""
    def display_domains(self):
        for row in self.domains:
            print(" ".join(str(st)for st in row))
        
    def get_set_from_column(self, col):
        nums = [self.grid[row][col] for row in range(9) if self.grid[row][col] != 0]
        return set(nums)
    
    def get_set_from_row(self, row):
        nums = [self.grid[row][col] for col in range(9) if self.grid[row][col] != 0]
        return set(nums)
    

    def get_set_from_block(self, block_start_row, block_start_col):
        nums = []
        for i in range(3):
            for j in range(3):
                num = self.grid[block_start_row + i][block_start_col + j]
                if num != 0:
                    nums.append(num)
        return set(nums)
    
    def get_indexes_non_empty_domain(self):
        indexes = []
        for row in range(9):
            indexes.extend([(row,col) for col in range(9) if self.domains[row][col] != None])

        return indexes
    
    def populate_random(self):
        for row in range(9):
            for col in range(9):
                if self.grid[row][col] == 0:
                    self.grid[row][col] = random.randint(1, 9)

    
    def available_indexes(self):
        available_indexes = []
        for row in range(9):
            for col in range(9):
                if self.grid[row][col] == 0:
                    available_indexes.append((row,col))
        return available_indexes
    
    def amount_of_comflicts(self):
        conflicts = 0
        for col in range(9):
            nums = [self.grid[row][col] for row in range(9) if self.grid[row][col] != 0]
            conflicts += len(nums) - len(set(nums))
        for row in self.grid:
            nums = [num for num in row if num != 0]
            conflicts += len(nums) - len(set(nums))

        for block_start_row in [0, 3, 6]:
            for block_start_col in [0, 3, 6]:
                nums = []
                for i in range(3):
                    for j in range(3):
                        num = self.grid[block_start_row + i][block_start_col + j]
                        if num != 0:
                            nums.append(num)
                conflicts += len(nums) -len(set(nums))

        return conflicts
        
        

                

            




"""if __name__ == "__main__":

    # Testing the SudokuPuzzle class
    puzzle = SudokuPuzzle('./Puzzles/Easy-P1.txt')
    puzzle.display()

    print("Valid Rows:", puzzle.is_valid_rows())
    print("Valid Columns:", puzzle.is_valid_columns())
    print("Valid Blocks:", puzzle.is_valid_blocks())
    print("Overall Valid:", puzzle.is_valid())

    # Failure case row column
    puzzle.set_digit(0, 0, 6)
    puzzle.display()
    print("Valid Rows:", puzzle.is_valid_rows())
    print("Valid Columns:", puzzle.is_valid_columns())
    print("Valid Blocks:", puzzle.is_valid_blocks())
    print("Overall Valid:", puzzle.is_valid())

    # Failure case blocks
    puzzle.set_digit(0, 1, 7)
    puzzle.display()
    print("Valid Blocks:", puzzle.is_valid_blocks())    
    print("Overall Valid:", puzzle.is_valid())

    # Output to file
    puzzle.output_to_file('./Puzzles/Output-Test.txt')

    # Set Domains
    puzzle.set_domains()

    # print Domains
    puzzle.display_domains()"""


puzzle = SudokuPuzzle('./Puzzles/Easy-P1.txt')

puzzle.populate_random()
puzzle.display()
print(puzzle.amount_of_comflicts())


6 1 8 3 5 6 3 7 1
7 3 4 2 2 3 6 1 9
7 9 5 4 2 8 8 5 6
6 1 7 4 2 9 5 4 3
4 4 9 9 6 7 1 9 7
7 5 2 3 1 6 9 4 4
1 3 5 1 9 4 6 2 3
4 2 1 4 9 6 3 8 6
3 8 9 8 3 4 4 8 4
68
